In [40]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from binance.client import Client
import datetime as dt
from pytz import timezone

In [ ]:
# 1. Data Retrieval

# Objective: Retrieve historical data using the API of a crypto exchange, such as Kraken or Binance. Requirements:
# • Select either BTCUSD or ETHUSD on Kraken, or BTCUSDT or ETHUSD on Binance as the trading pair. ☑️
# • Retrieve at least 30 days of 1-minute interval data for OHLC (Open, High, Low, Close) prices and trading volume. ☑️
# • Provide code that demonstrates the data retrieval process via the API. ☑️

#---------------------------------------------------------------------------------------------------------------------------------

api_key = 'hbJL4wO7LeYX6FkKuVW08PpoBgiOUOwdUYAhYLSG40xzpaywcEPgRjPgGBXGyS3B'
api_secret = 'qHm85rDfxljX5FhvjoAVEPbQ119Teu78SAw9KERQ3YtLD8mMHH2NRVUNef9Tb2zX'

client = Client(api_key, api_secret)


def get_historical_klines(symbol, interval, start_str, end_str=None):
    
    # Fetch historical klines from Binance.

    # param symbol: str e.g., 'BTCUSDT'
    # param interval: str e.g., Client.KLINE_INTERVAL_1HOUR
    # param start_str: str e.g., '7 Nov, 2024'
    # param end_str: str e.g., '6 Dec, 2024' (optional)
    # possible columns:  'open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades','taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    # return: pd.DataFrame
   
    klines = client.get_historical_klines(symbol, interval, start_str, end_str)
    df = pd.DataFrame(klines, columns=[
        'open_time', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df.set_index('open_time', inplace=True)
    # Keep only the Open, High, Low, Close columns and convert them to float
    df = df[['open', 'high', 'low', 'close']].astype(float)
    return df


#---------------------------------------------------------------------------------------------------------------------------------

#Run the program:

symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE  # 1-minute intervals
start_date = '7 Nov, 2024'
end_date = '6 Dec, 2024'

df = get_historical_klines(symbol, interval, start_date, end_date)

#save the dataframe in a csv format:

file_name = "Binance_BTCUSDT_30Days_1Min.csv" 
df.to_csv(file_name)


In [48]:
# 2. Data Cleaning

# Objective: Preprocess the raw data to make it suitable for analysis. Requirements:
# • Handle any missing timestamps or NaN values. ☑️
# • Remove duplicate rows, if any.
# • Address outliers in price or volume (explain your approach).
# • Convert timestamps to the CET timezone. ☑️
# • Aggregate the data into hourly intervals while maintaining the OHLC structure.


# Comment from Tim: 
# As my csv file from Task 1 already contains an ISO time, 
# I will use Matteo's csv files to demonstrate how I translate the time format from UNIX timestamp to ISO time.

#---------------------------------------------------------------------------------------------------------------------------------


import pandas as pd

def process_and_validate_file(filename):


    # Load the dataset
    data = pd.read_csv(filename)
    
    # Convert the 'time' column to a datetime object in UTC and then to CET
    data['time'] = pd.to_datetime(data['time'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Europe/Berlin')
    
    # Check for missing timestamps
    time_diff = data['time'].diff().value_counts()
    print(f"Timestamp Intervals in {filename}:")
    print(time_diff)
    
    if time_diff.shape[0] == 1 and time_diff.index[0] == pd.Timedelta(minutes=1):
        print("\nTimestamps are consistent (1-minute intervals).")
    else:
        print("\nTimestamps are not consistent. Please investigate further.")
    
    # Check for missing values (NaN) in the dataset
    missing_values = data.isna().sum()
    print(f"\nMissing Values in {filename}:")
    print(missing_values)
    
    if missing_values.sum() == 0:
        print("No missing values in the dataset.")
    else:
        print("There are missing values in the dataset. Please address them.")
    
    # Check for duplicate rows
    duplicate_count = data.duplicated().sum()
    print(f"\nNumber of duplicate rows in {filename}: {duplicate_count}")
    
    if duplicate_count > 0:
        print("Duplicate rows found. Consider removing them for a clean dataset.")
    else:
        print("No duplicate rows found.")
    
    # Optional: Drop duplicates (uncomment if needed)
    # data = data.drop_duplicates()
    
    return data


#---------------------------------------------------------------------------------------------------------------------------------

# Enter file name and run the program:
filename = 'BINANCE_BTCUSDT.csv'  # Assumes the file is in the current directory
processed_data = process_and_validate_file(filename)


Timestamp Intervals in BINANCE_BTCUSDT.csv:
time
0 days 00:01:00    3379
Name: count, dtype: int64

Timestamps are consistent (1-minute intervals).

Missing Values in BINANCE_BTCUSDT.csv:
time      0
open      0
high      0
low       0
close     0
Volume    0
dtype: int64
No missing values in the dataset.

Number of duplicate rows in BINANCE_BTCUSDT.csv: 0
No duplicate rows found.
